In [419]:
import torch
from torch import optim
import torchvision
from torchvision.transforms import v2
import torch.nn as nn
from PIL import Image
import pandas as pd
print(torchvision.__version__)
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

0.18.0+cu121


In [420]:
classes = ('anger', 'contempt', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise')

In [421]:
def preprocess(image):
    transform = v2.Compose([
        v2.ToImage() ,
        v2.ToDtype(torch.uint8, scale=True),
        v2.CenterCrop((96, 96)),
        v2.ToTensor(),
        v2.ToDtype(torch.float32, scale=True),
        v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = transform(image)
    return image

In [422]:
img = Image.open("data/archive/surprise/ffhq_238.png").convert('RGB')
img_tensor = preprocess(img)
img_tensor.shape


/home/hemangjain/Desktop/4th/FER/env/lib/python3.12/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


torch.Size([3, 96, 96])

In [423]:
def create_one_hot_encoding(label, classes):
    one_hot = torch.zeros(len(classes), dtype=torch.float32)
    
    if label in classes:
        index = classes.index(label)
        one_hot[index] = 1.0
    return one_hot

In [424]:
      
class CustomFERDataset(Dataset):
    def __init__(self, image_parent_directory, data_directory, transform=None):
        self.image_parent_directory = image_parent_directory
        self.df = pd.read_csv(data_directory)
        self.transform = transform
        self.classes = classes

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = self.image_parent_directory + self.df.iloc[idx, 0]
        image = Image.open(img_name).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        label = self.df.iloc[idx, 1]
        label_tensor = torch.tensor(create_one_hot_encoding(label, self.classes))
        
        return image, label_tensor

In [425]:
train_data = CustomFERDataset("data/archive/" , "data/train.csv" , transform=preprocess)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)


In [426]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.conv7 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv8 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.conv9 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.bn7 = nn.BatchNorm2d(128)
        self.bn8 = nn.BatchNorm2d(128)
        self.bn9 = nn.BatchNorm2d(128)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(128 * 12 * 12, 512)
        self.fc2 = nn.Linear(512, 8)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        
        residual = self.relu(self.bn7(self.conv7(x)))
        out = self.relu(self.bn8(self.conv8(residual)))
        out = self.bn9(self.conv9(out))
        out += residual
        x = self.pool(self.relu(out))
        
        x = x.view(-1, 128 * 12 * 12)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [427]:
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [428]:
def train(model, train_loader, criterion, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in tqdm(enumerate(train_loader)):
        # if batch_idx ==100:
        optimizer.zero_grad()
        output = model(data)
        # print(output.shape)
        # print(target)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

In [429]:
def test(model, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            if test_loss == 0:  # Only print for the first batch
                print(f"Data shape: {data.shape}")
                print(f"Target shape: {target.shape}")
                print(f"Output shape: {output.shape}")
                print(f"Output (first 5): {output[:5]}")
                print(f"Target (first 5): {target[:5]}")
            test_loss += criterion(output, target).item()  # sum up batch loss
            _,pred = output.max(1)
            _, target_labels = target.max(1)# get the index of the max log-probability
            correct += pred.eq(target_labels).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)\n')

In [430]:
test_data = CustomFERDataset("data/archive/" , "data/test.csv" , transform=preprocess)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False, num_workers=2)

In [431]:
num_epochs = 10
for epoch in range(1, num_epochs + 1):
    train(model, train_loader, criterion, optimizer, epoch)
    test(model, test_loader, criterion)

torch.save(model.state_dict(), "simple_cnn.pth")

0it [00:00, ?it/s]/home/hemangjain/Desktop/4th/FER/env/lib/python3.12/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(
/tmp/ipykernel_10951/1003653074.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label_tensor = torch.tensor(create_one_hot_encoding(label, self.classes))
/home/hemangjain/Desktop/4th/FER/env/lib/python3.12/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent u

Train Epoch: 1 [0/22540 (0%)]	Loss: 2.216792


101it [02:18,  1.60s/it]

Train Epoch: 1 [3200/22540 (14%)]	Loss: 2.113944


201it [04:50,  1.60s/it]

...

Train Epoch: 10 [19200/22540 (85%)]	Loss: 1.650325


701it [17:13,  1.51s/it]

Train Epoch: 10 [22400/22540 (99%)]	Loss: 1.403432


705it [17:19,  1.47s/it]
/home/hemangjain/Desktop/4th/FER/env/lib/python3.12/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(
/tmp/ipykernel_10951/1003653074.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label_tensor = torch.tensor(create_one_hot_encoding(label, self.classes))
/home/hemangjain/Desktop/4th/FER/env/lib/python3.12/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equiv

Data shape: torch.Size([32, 3, 96, 96])
Target shape: torch.Size([32, 8])
Output shape: torch.Size([32, 8])
Output (first 5): tensor([[-3.0439, -5.0921, -0.9990,  2.9183, -3.8426,  1.5390, -0.0827,  3.8359],
        [-1.7091, -5.6480, -0.3927,  4.1650, -4.5684,  1.1656,  0.4794,  3.8816],
        [-1.6520, -5.5303, -0.3717,  4.0633, -4.4947,  1.1420,  0.4782,  3.7922],
        [ 0.1515, -1.8115,  0.2933,  0.8518, -2.1661,  0.3968,  0.4395,  0.9688],
        [-6.3030,  1.9269, -0.1349, -4.9553,  4.2518, -4.4918, -3.0143, -0.9516]])
Target (first 5): tensor([[0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.]])

Test set: Average loss: 0.0436, Accuracy: 1360/2818 (48%)

